In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
'''for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))'''

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train = pd.read_csv('/kaggle/input/hms-harmful-brain-activity-classification/train.csv')
print(f'Train data shape = {train.shape}')
train.head()

In [ ]:
train[['patient_id', 'spectrogram_id', 'eeg_id']].value_counts()

In [ ]:
print(f'Count of unique patients: {len(train.patient_id.unique())}')
print(f'Count of unique spectrograms: {len(train.spectrogram_id.unique())}')
print(f'Count of unique EEGs: {len(train.eeg_id.unique())}')

In [ ]:
train_columns = ['eeg_id','eeg_sub_id','eeg_label_offset_seconds','spectrogram_id','spectrogram_sub_id','spectrogram_label_offset_seconds','label_id','patient_id']

In [ ]:
target_columns = ['seizure_vote','lpd_vote','gpd_vote','lrda_vote','grda_vote','other_vote']

In [ ]:
ex_patient_id = 2524
print(train[train.patient_id == ex_patient_id].eeg_id.unique())
print(f'Count EGG for one patient with id {ex_patient_id} is { len(train[train.patient_id == ex_patient_id].eeg_id.unique()) } ')

In [ ]:
train[train.patient_id == ex_patient_id]

In [ ]:
max(train.eeg_label_offset_seconds)

In [ ]:
3372.0/60

In [ ]:
train[train.eeg_label_offset_seconds == 3372.0]

In [ ]:
train[train.eeg_id == 2259539799]

In [ ]:
t1 = pd.read_parquet('/kaggle/input/hms-harmful-brain-activity-classification/train_eegs/2259539799.parquet')
print(t1.shape)
t1

In [ ]:
684400/200/60

In [ ]:
st1 = pd.read_parquet('/kaggle/input/hms-harmful-brain-activity-classification/train_spectrograms/1391458063.parquet')
print(st1.shape)
st1

In [ ]:
3971/60

**Conclusion of the first vision of data** 
<p>Each patient can have a lot of EEG. One spectrogram can belong the different EEG.

# Visualization EEG signals

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

### Parameters

In [ ]:
#All of the EEG data (for both train and test) was collected at a frequency of 200 samples per second
fr = 200 
eeg_example_id = 4045760041

In [ ]:
def get_eeg_data(eeg_id, fr = 200, dataset_path = '/kaggle/input/hms-harmful-brain-activity-classification'):
    eeg_path = dataset_path + "/train_eegs/{eeg_id}.parquet"
    eeg_data = pd.read_parquet(eeg_path.format(eeg_id=eeg_id))
    # add time column to the dataset
    eeg_data['time'] = eeg_data.index/fr
    eeg_data.set_index('time', inplace = True)
    
    return eeg_data

In [ ]:
eeg_data = get_eeg_data(eeg_example_id)
print(f' EEG data shape is {eeg_data.shape}')
eeg_data.head()

In [ ]:
print(f'Amount of seconds is {eeg_data.shape[0]/200}')

In [ ]:
def plot_eeg_signal(eeg_data, signal,interval = None):
    plt.figure(figsize=(20,10))
    sns.set_theme(style="whitegrid")
    if interval is None:
        sns.lineplot(x="time", y=signal, data=eeg_data)
    else:
        sns.lineplot(x="time", y=signal, data=eeg_data.loc[interval[0]: interval[1]])

In [ ]:
interval=(25.0,35.0)
eeg_data.loc[interval[0]: interval[1]]

In [ ]:
plot_eeg_signal(eeg_data, 'Fp1', interval)

In [ ]:
plot_eeg_signal(eeg_data, 'Fp1')

In [ ]:
def plot_eeg(eeg_data,interval = None):
    fig, axes = plt.subplots(nrows = eeg_data.shape[1], ncols =1, sharex=True, figsize=(30, 50))
    sns.set_theme(style="whitegrid")
    
    data = eeg_data if interval is None else eeg_data.loc[interval[0]: interval[1]]
    for i, col in enumerate(data.columns):
        ax = sns.lineplot(data=data, x='time', y=col, ax=axes[i])
        ax.set_title(col)
        ax.grid()        
    plt.subplots_adjust()

In [ ]:
plot_eeg(eeg_data, interval)

### Create the data for train of eeg and visualization them

In [ ]:
train.head()

In [ ]:
CHAINS = {
    'LL' : [("Fp1","F7"),("F7","T3"),("T3","T5"),("T5","O1")],
    'RL' : [("Fp2","F8"),("F8","T4"),("T4","T6"),("T6","O2")],
    'LP' : [("Fp1","F3"),("F3","C3"),("C3","P3"),("P3","O1")],
    'RP' : [("Fp2","F4"),("F4","C4"),("C4","P4"),("P4","O2")]
}
EGG_FR = 200
LABELED_SECS = 10

In [ ]:
EEG_PATH_TEMPL = '/kaggle/input/hms-harmful-brain-activity-classification/train_eegs/'
def get_train_row_data_eeg(eeg_df, row_index):
    data_row = eeg_df.iloc[row_index]
    eeg = pd.read_parquet(f'{EEG_PATH_TEMPL}{data_row.eeg_id}.parquet')
    eeg_offset = int(data_row.eeg_label_offset_seconds )
    eeg_offset += 20 # only 10 central seconds were labeled
    eeg = eeg.iloc[eeg_offset*EGG_FR:(eeg_offset+LABELED_SECS)*200]
    return (eeg, data_row[target_columns], data_row.expert_consensus)


In [ ]:
eeg1, label1, consensus = get_train_row_data_eeg(train,456)
eeg1.shape, label1, consensus

In [ ]:
def plot_chain(eeg_data, chain):
    fig, axes = plt.subplots(nrows = len(CHAINS), ncols =1, sharex=True, figsize=(20, 10))
    time_x = np.arange(-5,5,1.0/200)
    plt.xlim(-5.5, 5.5)
    x_ticks = np.arange(-5,5,1)
    for i, signals in enumerate(CHAINS[chain]):
        diff=eeg_data[signals[0]]-eeg_data[signals[1]]
        ax = sns.lineplot(x=time_x, y=diff, ax=axes[i])
        ax.set_title(f'{signals[0]} - {signals[1]}')
        ax.set_xticks(x_ticks)
        ax.grid(True)        
    plt.subplots_adjust()
    

In [ ]:
plot_chain(eeg1, "LP")

The train data of EEGs will be tensors with the following structure:
**eeg_data = [N, CH, DS ,T]**<br>
* **N** - number os samples
* **CH** = 4 - number of chains [ LL, RL, LP, RP] 
* **DS** = 4 - number of differences of signals {
* **T** = the length of labeled window *2000 = 200 friquence * 10 secs*

In [ ]:
EEG_PATH_TEMPL = '/kaggle/input/hms-harmful-brain-activity-classification/train_eegs/'
def get_train_data_eeg(eeg_df):
    
    data_row = eeg_df.iloc[row_index]
    eeg = pd.read_parquet(f'{EEG_PATH_TEMPL}{data_row.eeg_id}.parquet')
    eeg_offset = int(data_row.eeg_label_offset_seconds )
    eeg_offset += 20 # only 10 central seconds were labeled
    eeg = eeg.iloc[eeg_offset*EGG_FR:(eeg_offset+LABELED_SECS)*200]
    return (eeg, data_row[target_columns], data_row.expert_consensus)

In [ ]:
len(train.eeg_id.unique())

In [ ]:
TARGET_COLUMNS = target_columns
EEG_CH = 4
EEG_DS = 4
EEG_T = 2000

In [ ]:
eeg_groups = train.head(300).groupby('eeg_id')

egg_chains_data = []
# iterate over each group
for eeg_id, df_group in eeg_groups:
    #print('\n Handling egg with id {}('.format(egg_id))
    eeg_parquet = pd.read_parquet(f'{EEG_PATH_TEMPL}{eeg_id}.parquet')
    
    for row_index, row in df_group.iterrows():
        eeg_offset = int(row.eeg_label_offset_seconds )+ 20 # only 10 central seconds from 50 secs were labeled
        eeg = eeg_parquet.iloc[eeg_offset*EGG_FR:(eeg_offset+LABELED_SECS)*200]
        cur_row = np.zeros(shape = (EEG_CH,EEG_DS,EEG_T))
        for ch_i, chain in enumerate(CHAINS):
            for s_i, signals in enumerate(CHAINS[chain]):
                diff=eeg[signals[0]]-eeg[signals[1]]
                cur_row[ch_i][s_i] = diff
        egg_chains_data.append(cur_row)

In [ ]:
np.array(egg_chains_data).shape

In [ ]:
GET_ROW = 0
EEG_PATH = 'train_eegs/'
SPEC_PATH = 'train_spectrograms/'

train = pd.read_csv('train.csv')
row = train.iloc[GET_ROW]

eeg = pd.read_parquet(f'{EEG_PATH}{row.eeg_id}.parquet')
eeg_offset = int( row.eeg_label_offset_seconds )
eeg = eeg.iloc[eeg_offset*200:(eeg_offset+50)*200]

spectrogram = pd.read_parquet(f'{SPEC_PATH}{row.spectrogram_id}.parquet')
spec_offset = int( row.spectrogram_label_offset_seconds )
spectrogram = spectrogram.loc[(spectrogram.time>=spec_offset)
                     &(spectrogram.time<spec_offset+600)]

## Visualization spectrograms

In [ ]:
test_egg = pd.read_parquet('/kaggle/input/hms-harmful-brain-activity-classification/test_eegs/3911565283.parquet')
print(test_egg.shape)
test_egg

In [ ]:
test_spectro = pd.read_parquet('/kaggle/input/hms-harmful-brain-activity-classification/test_spectrograms/853520.parquet')
print(test_spectro.shape)
test_spectro

In [ ]:
df_spectogram = pd.read_parquet('/kaggle/input/hms-harmful-brain-activity-classification/train_spectrograms/1000189855.parquet')
print("spectogram record shape: ", df_spectogram.shape)
df_spectogram.head()